<a href="https://colab.research.google.com/github/19A81A0487/Computer-vision/blob/main/Mobilenet(computer_vision).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Packages

In [ ]:
!pip install onnx
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.4 MB/s eta 0:00:00


#Model

In [ ]:
import onnx
model_path = "/content/drive/MyDrive/mobilenetv2-7 (1).onnx"

model = onnx.load(model_path)

#Required libraries

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (24.4 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debco

#Video processing frame by frame

In [ ]:
import cv2
import numpy as np
import onnxruntime as ort
from google.colab.patches import cv2_imshow
import pytesseract


onnx_model_path = "/content/drive/MyDrive/mobilenetv2-7 (1).onnx"
ort_session = ort.InferenceSession(onnx_model_path)

def process_output(output, frame):

    probabilities = np.exp(output) / np.sum(np.exp(output), axis=1, keepdims=True)

    predicted_class = np.argmax(probabilities, axis=1)

    label = f"Predicted class: {predicted_class[0]}"


    processed_frame = frame.copy()
    cv2.putText(processed_frame, label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    return processed_frame



video_path = "/content/project_video.com).mp4"
cap = cv2.VideoCapture(video_path)


fps = cap.get(cv2.CAP_PROP_FPS)

frame_count = 0


while cap.isOpened() and frame_count < 50:
    ret, frame = cap.read()
    if not ret:
        break



    print(f"Frame {frame_count + 1}: ")

    input_size = (224, 224)
    preprocessed_frame = cv2.resize(frame, input_size)
    preprocessed_frame = np.transpose(preprocessed_frame, (2, 0, 1))
    preprocessed_frame = np.expand_dims(preprocessed_frame, axis=0)
    preprocessed_frame = preprocessed_frame.astype(np.float32) / 255.0

    input_name = ort_session.get_inputs()[0].name
    output_name = ort_session.get_outputs()[0].name
    outputs = ort_session.run([output_name], {input_name: preprocessed_frame})
    processed_frame = process_output(outputs[0], frame)


    cv2_imshow(processed_frame)


    frame_count += 1


    delay = int(1000 / fps)


    if cv2.waitKey(delay) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


print(f"Frame rate (FPS): {fps}")
print("Displayed first 50 frames of the video.")


# text recognition and time steps

In [ ]:
import cv2
import numpy as np
import onnxruntime as ort
from google.colab.patches import cv2_imshow
import pytesseract


onnx_model_path = "/content/drive/MyDrive/mobilenetv2-7 (1).onnx"
ort_session = ort.InferenceSession(onnx_model_path)


def process_output(output, frame):

    probabilities = np.exp(output) / np.sum(np.exp(output), axis=1, keepdims=True)


    predicted_class = np.argmax(probabilities, axis=1)


    label = f"Predicted class: {predicted_class[0]}"


    processed_frame = frame.copy()
    cv2.putText(processed_frame, label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    return processed_frame

def perform_ocr(frame):

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


    _, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    text = pytesseract.image_to_string(threshold)

    return text


video_path = "/content/project_video.com).mp4"
cap = cv2.VideoCapture(video_path)


fps = cap.get(cv2.CAP_PROP_FPS)

frame_count = 0


while cap.isOpened() and frame_count < 50:  # Process first 50 frames
    ret, frame = cap.read()
    if not ret:
        break

    print(f"Frame {frame_count + 1}: ")

    input_size = (224, 224)
    preprocessed_frame = cv2.resize(frame, input_size)
    preprocessed_frame = np.transpose(preprocessed_frame, (2, 0, 1))  # Channels first
    preprocessed_frame = np.expand_dims(preprocessed_frame, axis=0)  # Add batch dimension
    preprocessed_frame = preprocessed_frame.astype(np.float32) / 255.0  # Normalize to [0, 1]
    input_name = ort_session.get_inputs()[0].name
    output_name = ort_session.get_outputs()[0].name
    outputs = ort_session.run([output_name], {input_name: preprocessed_frame})
    processed_frame = process_output(outputs[0], frame)


    text = perform_ocr(processed_frame)


    print("Extracted text:", text)

    timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
    print("Timestamp:", timestamp)

    # Display the processed frame
    cv2_imshow(processed_frame)

    # Increment frame count
    frame_count += 1

    # Calculate delay based on FPS (delay in milliseconds)
    delay = int(1000 / fps)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(delay) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Print information about frame rate
print(f"Frame rate (FPS): {fps}")

